In [4]:
from statsbombpy import sb
import pandas as pd
import networkx as nx
import networkx as nx
from itertools import combinations
import multiprocessing

from networkx.algorithms.isomorphism import DiGraphMatcher
multiprocessing.set_start_method('fork', force=True)



matches = sb.matches(competition_id=9, season_id=281)
match_ids = matches['match_id'].values.tolist()


#All events sorted for barca home games and possession 
#find ids for barca home matches - only home games because then locations can be compared between different games
events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024",
    gender="male"
)
events = events[events['match_id'].isin(match_ids)]
df = events[events["team"]=="Bayer Leverkusen"]
df = df[df["possession_team"]=="Bayer Leverkusen"]
#filter threshold for Xg:
df_xg = df[~df['shot_statsbomb_xg'].between(0, 0.05)]
#Events sorted in a specific order so each passing sequence is correctly sorted
sequences_sorted = df_xg.sort_values(['match_id', 'period','timestamp'], ascending=[True, True, True])
#make new ids because right now there is ids from 1 to x for each match but it repeats from 1 and up in every match so each possession id points to different matches 
# - i just put the possession id after match_id in the newly created id
sequences_sorted['possession_id'] = sequences_sorted['match_id'].astype(str) + sequences_sorted['possession'].astype(str)
sequences_sorted['possession_id'] = sequences_sorted['possession_id'].astype(int)
#get the ids of sequences which contain a shot (contain an xg value)
shot_sequences = sequences_sorted[sequences_sorted["shot_statsbomb_xg"].notna()]
shot_sequences_ids = shot_sequences["possession_id"].unique()
#filter for possession sequences which end with a shot
sequences_filtered = sequences_sorted[sequences_sorted['possession_id'].isin(shot_sequences_ids)]
#fill all rows with an xg for the corresponding sequence - right now there are many missing values in "shot_statsbomb_xg"
sequences_filtered['xg'] = sequences_filtered.groupby('possession_id')['shot_statsbomb_xg'].transform(lambda group: group.fillna(method='ffill').fillna(method='bfill'))
#now we dont need the shot event rows any more so remove them
sequences_filtered = sequences_filtered[sequences_filtered["type"]!="Shot"]
#filter the df to only include row with an id the of a pass recipient and we subset the columns
player_final_sequences =  sequences_filtered[sequences_filtered["pass_recipient"].notna()][["player_id", "pass_recipient_id", "possession_id", "xg","timestamp"]]
player_final_sequences
player_final_sequences = player_final_sequences.sort_values(['possession_id', 'timestamp'], ascending=[True, True])
player_final_sequences = player_final_sequences[player_final_sequences['player_id'] != player_final_sequences['pass_recipient_id']]
player_final_sequences['sequence'] = player_final_sequences.groupby('possession_id').cumcount(ascending=False) + 1
player_final_sequences
#remove sequences with few passes if wanted
index_counts = player_final_sequences['possession_id'].value_counts()
player_final_sequences = player_final_sequences[player_final_sequences['possession_id'].isin(index_counts[index_counts > 5].index)]
possession_index = player_final_sequences["possession_id"].unique()


#iterate over possession ids and each row and append edges to a list for each graph and append that graph to a graphs dictionary (directed graph created with "nx.DiGraph(edges)")
#xg added as an attribute for each graph
graphs_dict = {}
for j in possession_index:
    edges = []
    for i in player_final_sequences.index:
        if j == player_final_sequences["possession_id"][i]:
            edge = (player_final_sequences["player_id"][i], player_final_sequences["pass_recipient_id"][i])
            edges.append(edge)
            if j not in graphs_dict:
                graphs_dict[j] = {"xg": player_final_sequences["xg"][i], "graph": None}
            else:
                graphs_dict[j]["xg"] = player_final_sequences["xg"][i]

    graph = nx.DiGraph()
    graph.add_edges_from(edges)

    # Add sequence as an edge attribute
    for i in player_final_sequences.index:
        if j == player_final_sequences["possession_id"][i]:
            graph[player_final_sequences["player_id"][i]][player_final_sequences["pass_recipient_id"][i]]['sequence'] = player_final_sequences["sequence"][i]

    graphs_dict[j]["graph"] = graph


/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/var/folders/dl/s9k442mn7m1077_4b28hhf6c0000gn/T/ipykernel_89834/1609860519.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sequences_filtered['xg'] = sequences_filtered.groupby('possession_id')['shot_statsbomb_xg'].transform(lambda group: group.fillna(method='ffill').fillna(method='bfill'))


In [5]:
def frequent_singletons(min_sup, edge_matrix, total_graphs):
    items_counted = {}
    edge_attributes = {}

    for idx, edge_list in enumerate(edge_matrix):
        for edge in edge_list:

            edge_key = (edge[0], edge[1])
            items_counted[edge_key] = items_counted.get(edge_key, 0) + 1
            
            if edge_key not in edge_attributes:
                edge_attributes[edge_key] = {**edge[2], 'source': idx}

    F = []
    for key, value in items_counted.items():
        support_percentage = (value / total_graphs) * 100  
        if support_percentage >= min_sup:
            F.append(key)

    
    F_graphs = []
    for edge in F:
        g = nx.DiGraph()

        g.add_edge(edge[0], edge[1], **edge_attributes[edge])
        F_graphs.append(g)
    
    return F_graphs

def generate_candidates(F, k):
    
    candidates = set()

    for g1, g2 in combinations(F, 2):

        edge_set_g1 = tuple(sorted(g1.edges(data=True)))
        edge_set_g2 = tuple(sorted(g2.edges(data=True)))
        
        source_g1 = next((edge[2].get('source') for edge in edge_set_g1 if 'source' in edge[2]), None)
        source_g2 = next((edge[2].get('source') for edge in edge_set_g2 if 'source' in edge[2]), None)

        if source_g1 != source_g2:
            continue

        edges_g1 = sorted(g1.edges(data=True), key=lambda e: e[2].get('sequence', 0))
        edges_g2 = sorted(g2.edges(data=True), key=lambda e: e[2].get('sequence', 0))
        
        if (abs(edges_g1[0][2].get('sequence') - edges_g2[0][2].get('sequence')) == 1
            or abs(edges_g2[0][2].get('sequence') - edges_g1[0][2].get('sequence')) == 1
            ):
            
            union_graph = nx.compose(g1, g2)
            
            if union_graph.number_of_edges() == k:  
                candidates.add(union_graph)
    
    return candidates



def count_support(C, graph_db):
    F_count = {}
    
    def edge_match(attr1, attr2):
        return attr1.get('sequence') == attr2.get('sequence')
    
    for graph in graph_db:
        for candidate in C:
            GM = DiGraphMatcher(graph, candidate, edge_match=edge_match)
            if GM.subgraph_is_isomorphic():  
                F_count[candidate] = F_count.get(candidate, 0) + 1
                
    return F_count

# Filter frequent candidates based on minimum support (in percentage)
def filter_frequent(F_count, min_sup, total_graphs):
    return [
        key for key, value in F_count.items()
        if (value / total_graphs) * 100 >= min_sup
    ]
# main function
def apriori_graph_mining(min_sup, edge_matrix, graph_db, max_k):
    frequent_total = []
    results = []
    total_graphs = len(graph_db)  

    F = frequent_singletons(min_sup, edge_matrix, total_graphs)

    frequent_total.extend(F)

    k = 2  
    while k <= max_k:
        print(f"\nIteration {k}:")

        
        C = generate_candidates(F, k)
        F_count = count_support(C, graph_db)
        F = filter_frequent(F_count, min_sup, total_graphs)

        if not F: 
            break

        frequent_total.extend(F)

        for subgraph in F:
            support_count = F_count[subgraph]
            support_percentage = (support_count / total_graphs) * 100

            edges_with_attrs = [
                (u, v, attr) for u, v, attr in subgraph.edges(data=True)
            ]
            results.append(
                {
                    "k": k,
                    "edges": edges_with_attrs,
                    "support_percentage": support_percentage, 
                }
            )

        k += 1

    results_df = pd.DataFrame(results)
    return frequent_total, results_df

graph_list = [value["graph"] for value in graphs_dict.values()]

edge_matrix = [list(graph.edges(data=True)) for graph in graph_list]
GRAPH_DB = graph_list  
frequent_subgraphs, patterns_df = apriori_graph_mining(5, edge_matrix, GRAPH_DB, 7)




Iteration 2:

Iteration 3:

Iteration 4:


In [6]:
filtered_df = patterns_df[patterns_df['edges'].apply(
    lambda edges: any('sequence' in attr and attr['sequence'] == 1 for _, _, attr in edges)
)]
filtered_df

,k,edges,support_percentage
3,2,"[(40724.0, 32712.0, {'sequence': 2, 'source': ...",51.871658
4,2,"[(10336.0, 32289.0, {'sequence': 1, 'source': ...",51.871658
7,2,"[(28268.0, 40724.0, {'sequence': 2, 'source': ...",51.871658
22,3,"[(28268.0, 40724.0, {'sequence': 2, 'source': ...",25.133690


In [2]:
events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024",
    gender="male"
)

df = match_ids(events, "Bayer Leverkusen", season_id=281, competition_id=9)
possesion, final_sequence = create_graphs(df)
graph_list, graph_dict = create_graphs_dict(possesion, final_sequence)


graph_list_sample = graph_list

# Create a list of edges from the sampled graph_list
edge_matrix = [list(graph.edges()) for graph in graph_list_sample]
GRAPH_DB = graph_list_sample  # List of graphs in the database
min_sup = 0

for i, graph in enumerate(graph_list_sample):
    print(f"\nGraph {i+1} with sequence attributes:")
    
    # Print Edge Attributes
    print("\nEdges and their sequence attributes:")
    for u, v, attr in graph.edges(data=True):
        print(f"Edge {u} -> {v}, Attributes: {attr}")



/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warning


Graph 1 with sequence attributes:

Edges and their sequence attributes:
Edge 4.3 -> 4.2, Attributes: {'sequence': 50}
Edge 4.3 -> 5.2, Attributes: {'sequence': 37}
Edge 4.3 -> 3.4, Attributes: {'sequence': 20}
Edge 4.2 -> 3.1, Attributes: {'sequence': 49}
Edge 4.2 -> 4.1, Attributes: {'sequence': 42}
Edge 4.2 -> 4.3, Attributes: {'sequence': 21}
Edge 4.2 -> 4.4, Attributes: {'sequence': 35}
Edge 3.1 -> 4.1, Attributes: {'sequence': 26}
Edge 3.1 -> 4.2, Attributes: {'sequence': 43}
Edge 4.1 -> 5.1, Attributes: {'sequence': 3}
Edge 4.1 -> 3.2, Attributes: {'sequence': 45}
Edge 4.1 -> 5.0, Attributes: {'sequence': 41}
Edge 4.1 -> 4.2, Attributes: {'sequence': 39}
Edge 4.1 -> 4.0, Attributes: {'sequence': 25}
Edge 5.1 -> 4.1, Attributes: {'sequence': 4}
Edge 5.1 -> 5.2, Attributes: {'sequence': 2}
Edge 3.2 -> 3.1, Attributes: {'sequence': 27}
Edge 3.2 -> 3.3, Attributes: {'sequence': 29}
Edge 5.0 -> 4.1, Attributes: {'sequence': 40}
Edge 5.0 -> 5.1, Attributes: {'sequence': 11}
Edge 5.2 -

In [3]:

import networkx as nx
from itertools import combinations
from networkx.algorithms.isomorphism import DiGraphMatcher

def frequent_singletons(min_sup, edge_matrix):
    items_counted = {}
    
    for edge_list in edge_matrix:
        for edge in edge_list:
            # Use only source and target nodes for counting
            edge_key = (edge[0], edge[1])
            items_counted[edge_key] = items_counted.get(edge_key, 0) + 1
    
    # Filter edges that meet the min_sup
    F = [key for key, value in items_counted.items() if value >= min_sup]
    
    F_graphs = []
    for edge in F:
        g = nx.DiGraph()
        g.add_edge(edge[0], edge[1])  # Add edge without sequence for the subgraph
        F_graphs.append(g)
    
    return F_graphs

# Step 1: Find frequent singletons (edges)
F = frequent_singletons(0, edge_matrix)
k = 2

# Generate candidates of size k subgraphs from frequent subgraphs
def generate_candidates(F, k):
    candidates = set()
    
    # Iterate over all pairs of frequent subgraphs (F)
    for g1, g2 in combinations(F, 2):

        edges_g1 = sorted(g1.edges(data=True), key=lambda e: e[2].get('sequence', 0))
        edges_g2 = sorted(g2.edges(data=True), key=lambda e: e[2].get('sequence', 0))
        
        if edges_g1[2].get('sequence') > edges_g2[2].get('sequence'):
            union_graph = nx.compose(g1, g2)
            
            # Ensure that the union has the correct number of edges
            if union_graph.number_of_edges() == k:  # Check edge size instead of node size
                candidates.add(union_graph)
    
    return candidates

# Count the support for each candidate in the graph database
def count_support(C, graph_db):
    F_count = {}
    for graph in graph_db:
        for candidate in C:
            GM = DiGraphMatcher(graph, candidate)
            if GM.subgraph_is_isomorphic():  # Check for subgraph isomorphism
                F_count[candidate] = F_count.get(candidate, 0) + 1
    return F_count

# Filter frequent candidates based on minimum support
def filter_frequent(F_count, min_sup):
    return [key for key, value in F_count.items() if value >= min_sup]

# Main function to run the apriori graph mining algorithm
def apriori_graph_mining(min_sup, edge_matrix, graph_db, max_k):
    frequent_total = []
    
    # Step 1: Find frequent singletons (edges)
    F = frequent_singletons(min_sup, edge_matrix)
    
    # Add initial frequent items to the total list
    frequent_total.extend(F)
    
    k = 2  # Start with size-2 subgraphs
    while k <= max_k:
        print(f"\nIteration {k}:")
        
        # Step 2: Generate candidate subgraphs of size k
        C = generate_candidates(F, k)
        
        # Step 3: Count support for each candidate in the graph database
        F_count = count_support(C, graph_db)
        
        # Step 4: Filter out frequent candidates that meet the minimum support
        F = filter_frequent(F_count, min_sup)
        
        if not F:  # If no frequent candidates are found, stop the algorithm
            print(f"No frequent subgraphs found for size {k}. Terminating.")
            break
        
        # Add frequent items to the total list
        frequent_total.extend(F)
        
        print(f"Frequent subgraphs of size {k}:")
        for subgraph in F:
            for u, v, attr in subgraph.edges(data=True):
                print(f"Edge {u} -> {v}, Attributes: {attr}")
        
        k += 1  # Move to the next size of subgraphs

    return frequent_total



# Illustration af hver gennemgang af algoritmen

In [4]:
F = frequent_singletons(2, edge_matrix)
C = generate_candidates(F, 2)
count = count_support(C, GRAPH_DB)
filter = filter_frequent(count, 2)

print(C)

IndexError: list index out of range

In [5]:
edge_matrix

[[(4.3, 4.2),
  (4.3, 5.2),
  (4.3, 3.4),
  (4.2, 3.1),
  (4.2, 4.1),
  (4.2, 4.3),
  (4.2, 4.4),
  (3.1, 4.1),
  (3.1, 4.2),
  (4.1, 5.1),
  (4.1, 3.2),
  (4.1, 5.0),
  (4.1, 4.2),
  (4.1, 4.0),
  (5.1, 4.1),
  (5.1, 5.2),
  (3.2, 3.1),
  (3.2, 3.3),
  (5.0, 4.1),
  (5.0, 5.1),
  (5.2, 4.2),
  (5.2, 6.4),
  (4.4, 3.3),
  (3.3, 4.3),
  (3.3, 3.2),
  (3.3, 2.3),
  (3.3, 2.2),
  (3.4, 3.3),
  (3.4, 2.4),
  (4.0, 6.1),
  (6.1, 5.1),
  (2.4, 2.3),
  (2.3, 3.3),
  (2.2, 6.0),
  (6.0, 5.0)],
 [(4.0, 3.1),
  (3.1, 3.2),
  (3.1, 3.0),
  (3.2, 4.4),
  (3.2, 3.1),
  (4.4, 3.3),
  (3.3, 3.2),
  (3.0, 5.1),
  (5.1, 5.2)],
 [(2.2, 2.3),
  (2.3, 2.4),
  (2.3, 3.3),
  (2.4, 1.3),
  (1.3, 1.2),
  (1.3, 2.3),
  (1.2, 1.1),
  (1.1, 1.3),
  (3.3, 4.1),
  (4.1, 5.1),
  (5.1, 6.1),
  (6.1, 6.2)],
 [(6.0, 5.1),
  (5.1, 5.2),
  (5.1, 6.1),
  (5.2, 4.3),
  (5.2, 2.1),
  (4.3, 5.2),
  (2.1, 3.1),
  (3.1, 4.2),
  (3.1, 4.0),
  (3.1, 4.1),
  (4.2, 3.1),
  (4.0, 1.1),
  (1.1, 2.3),
  (2.3, 3.3),
  (3.3, 3.1),
  (

In [6]:
C2 = generate_candidates(C, 3)
count2 = count_support(C2, GRAPH_DB)
filter2 = filter_frequent(count2, 2)

for i in filter2:
    print(i.edges())

NameError: name 'C' is not defined

In [13]:
nx.draw_networkx(filter2[0])

IndexError: list index out of range

In [18]:
C3 = generate_candidates(C2, 4)
count3 = count_support(C3, GRAPH_DB)
filter3 = filter_frequent(count3, 2)

# Illustration af algoritmen på en gang

In [7]:
# test
frequents = apriori_graph_mining(5, edge_matrix, GRAPH_DB, 5)
for pattern in frequents:
    print(pattern.edges())


Iteration 2:


IndexError: list index out of range